# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('users_behavior.csv')
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


сalls — количество звонков,<br>
minutes — суммарная длительность звонков в минутах,<br>
messages — количество sms-сообщений,<br>
mb_used — израсходованный интернет-трафик в Мб,<br>
is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [ ]:
df['messages'] = df['messages'].astype('int')

In [ ]:
df['calls'] = df['calls'].astype('int')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


изменила тип данных у количества званков и сообщений на int, так как они являются целыми числами

## Разбейте данные на выборки

In [ ]:
features = df.drop(['is_ultra'], axis=1)
features.head()

,calls,minutes,messages,mb_used
0,40,311.90,83,19915.42
1,85,516.75,56,22696.96
2,77,467.66,86,21060.45
3,106,745.53,81,8437.39
4,66,418.74,1,14502.75


In [ ]:
target = df['is_ultra']
target.head()

,is_ultra
0,0
1,0
2,0
3,1
4,0


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
features_full_train, features_test, target_full_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [ ]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features_full_train, target_full_train, test_size=0.25, random_state=12345)

In [ ]:
features_train.shape, features_valid.shape, features_test.shape

((1807, 4), (603, 4), (804, 4))

## Исследуйте модели

Дерево решений

In [ ]:
from sklearn.tree import DecisionTreeClassifier
best_model = None
best_result = 0
count_deep = 0
for i in range(1, 6):
  model = DecisionTreeClassifier(random_state = 12345, max_depth=i)
  model.fit(features_train, target_train)
  result = model.score(features_valid, target_valid)
  if result > best_result:
        best_model = model
        best_result = result
        count_deep = i
print(f'Accuracy наилучшей модели дерева решений (с глубиной:{count_deep}) на валидационной выборке составило:{best_result}')

Accuracy наилучшей модели дерева решений (с глубиной:3) на валидационной выборке составило:0.7943615257048093


Случайный лес

In [ ]:
from sklearn.ensemble import RandomForestClassifier
best_model = None
best_result = 0
count_tree = 0
for est in range(1, 70):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result = result
        count_tree = est
print(f'Accuracy наилучшей модели случайного леса (с количеством деревьев:{count_tree}) на валидационной выборке составило:{best_result}')

Accuracy наилучшей модели случайного леса (с количеством деревьев:20) на валидационной выборке составило:0.8275290215588723


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [ ]:
df['messages'] = df['messages'].astype('int')

In [ ]:
df['calls'] = df['calls'].astype('int')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


изменила тип данных у количества званков и сообщений на int, так как они являются целыми числами

## Разбейте данные на выборки

In [ ]:
features = df.drop(['is_ultra'], axis=1)
features.head()

,calls,minutes,messages,mb_used
0,40,311.90,83,19915.42
1,85,516.75,56,22696.96
2,77,467.66,86,21060.45
3,106,745.53,81,8437.39
4,66,418.74,1,14502.75


In [ ]:
target = df['is_ultra']
target.head()

,is_ultra
0,0
1,0
2,0
3,1
4,0


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
features_full_train, features_test, target_full_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [ ]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features_full_train, target_full_train, test_size=0.25, random_state=12345)

In [ ]:
features_train.shape, features_valid.shape, features_test.shape

((1807, 4), (603, 4), (804, 4))

## Исследуйте модели

Дерево решений

In [ ]:
from sklearn.tree import DecisionTreeClassifier
best_model = None
best_result = 0
count_deep = 0
for i in range(1, 6):
  model = DecisionTreeClassifier(random_state = 12345, max_depth=i)
  model.fit(features_train, target_train)
  result = model.score(features_valid, target_valid)
  if result > best_result:
        best_model = model
        best_result = result
        count_deep = i
print(f'Accuracy наилучшей модели дерева решений (с глубиной:{count_deep}) на валидационной выборке составило:{best_result}')

Accuracy наилучшей модели дерева решений (с глубиной:3) на валидационной выборке составило:0.7943615257048093


Случайный лес

In [ ]:
from sklearn.ensemble import RandomForestClassifier
best_model = None
best_result = 0
count_tree = 0
for est in range(1, 70):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result = result
        count_tree = est
print(f'Accuracy наилучшей модели случайного леса (с количеством деревьев:{count_tree}) на валидационной выборке составило:{best_result}')

Accuracy наилучшей модели случайного леса (с количеством деревьев:20) на валидационной выборке составило:0.8275290215588723


Логистическая регрессия

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid)
print(f'Accuracy наилучшей модели логистической регрессии на валидационной выборке составило:{result}')

Accuracy наилучшей модели логистической регрессии на валидационной выборке составило:0.7412935323383084


Наилучшей моделью оказался случайный лес, так как имеет наибольшую метрику accuracy и более устойчмва к переобучению, чем дерево решений

## Проверьте модель на тестовой выборке

In [ ]:
model = RandomForestClassifier(random_state=12345, n_estimators=20)
model.fit(features_train, target_train)
result = model.score(features_test, target_test)
result

0.7835820895522388

Логистическая регрессия

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid)
print(f'Accuracy наилучшей модели логистической регрессии на валидационной выборке составило:{result}')

Accuracy наилучшей модели логистической регрессии на валидационной выборке составило:0.7412935323383084


Наилучшей моделью оказался случайный лес, так как имеет наибольшую метрику accuracy и более устойчмва к переобучению, чем дерево решений

## Проверьте модель на тестовой выборке

In [ ]:
model = RandomForestClassifier(random_state=12345, n_estimators=20)
model.fit(features_train, target_train)
result = model.score(features_test, target_test)
print(f'Accuracy модели случайного леса (с количеством деревьев:{count_tree}) на тестовой выборке составило:{result}')

Accuracy модели случайного леса (с количеством деревьев:20) на тестовой выборке составило:0.7835820895522388


На тестовой выборке модель показала хорошие результаты accuracy = 0,78 это означает, что я подобрала хорошую модель(случайный лес) с правильными гиперпараметрами(20 деревьевЪ

## (бонус) Проверьте модели на адекватность

Проверим раюотоспособность модели на очевидных данных

In [ ]:
X_easy_test = pd.DataFrame({
    'calls': [150, 30],
    'minutes': [900, 200],
    'messages': [20, 5],
    'mb_used': [30000, 10000]
})

y_easy_test = [1, 0]

easy_predictions = model.predict(X_easy_test)
print(f'Предсказания на очевидных данных:{easy_predictions}')
print(f'Ожидаемые ответы:{y_easy_test}')

Предсказания на очевидных данных:[1 0]
Ожидаемые ответы:[1, 0]


**Вывод:**<br>
Построенная модель случайного леса с 20 деревьями демонстрирует хорошее качество предсказаний (78% точности) и может быть рекомендована для использования в бизнес-задачах оператора для прогнозирования выбора тарифа новыми клиентами<br>
Модель успешно прошла проверку на тестовых данных и показала логичные результаты на очевидных примерах, что подтверждает ее надежность.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
